In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
#convert stata data to pandas
stata_data='../NSDUH_2017.DTA'
df=pd.read_stata(stata_data)
t=df.shape[0]

In [3]:
dfa=df[df['CATAG2']!='1 - 12-17 Years Old']
dfa = dfa[df.suicplan != '85 - BAD DATA Logically assigned']
dfa = dfa[df.suicplan != '94 - DON T KNOW']
dfa = dfa[df.suicplan != '97 - REFUSED']
dfa = dfa[df.suicplan != '98 - BLANK']

/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [4]:
dfa.groupby(['suicplan']).size()

suicplan
1 - Yes                               845
2 - No                               1740
85 - BAD DATA Logically assigned        0
94 - DON T KNOW                         0
97 - REFUSED                            0
98 - BLANK                              0
99 - LEGITIMATE SKIP                39652
dtype: int64

In [5]:
dfa.loc[df.suicplan=='1 - Yes', 'suicide'] = 'plan'
dfa.loc[df.suicplan=='2 - No', 'suicide'] = 'thought'
dfa.loc[df.suicplan=='99 - LEGITIMATE SKIP', 'suicide'] = 'no'

In [6]:
dfa.groupby('irpnrnmrec').size()

irpnrnmrec
1 - Within the past 30 days                               648
2 - More than 30 days ago but within the past 12 mos     1596
3 - More than 12 months ago                              2953
9 - NEVER MISUSED PAIN RELIEVERS                        37040
dtype: int64

In [7]:
dfa.loc[df.irpnrnmrec=='1 - Within the past 30 days', 'misuse'] = 'last year'
dfa.loc[df.irpnrnmrec=='2 - More than 30 days ago but within the past 12 mos', 'misuse'] = 'last year'
dfa.loc[df.irpnrnmrec=='3 - More than 12 months ago', 'misuse'] = 'sometime'
dfa.loc[df.irpnrnmrec=='9 - NEVER MISUSED PAIN RELIEVERS', 'misuse'] = 'never'

In [8]:
gf=dfa.groupby(['suicide','misuse']).size()

In [9]:
gf.unstack()[['never','sometime','last year']]

misuse,never,sometime,last year
suicide,,,
no,35182,2642,1828
plan,563,104,178
thought,1295,207,238


In [58]:
y=dfa.suicide
dummies = pd.get_dummies(y)
y=dummies['no']

In [60]:
X=dfa.misuse
X = pd.get_dummies(X)

In [61]:
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.058703
         Iterations 7


/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/Users/mahdimoqri/miniconda3/envs/myenv/lib/python3.7/site-packages/statsmodels/discrete/discrete_model.py:3313: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.llf/self.llnull


                         Results: Logit
Model:              Logit            Pseudo R-squared: inf      
Dependent Variable: no               AIC:              4964.8495
Date:               2019-02-18 16:51 BIC:              4990.8027
No. Observations:   42237            Log-Likelihood:   -2479.4  
Df Model:           2                LL-Null:          0.0000   
Df Residuals:       42234            LLR p-value:      1.0000   
Converged:          1.0000           Scale:            1.0000   
No. Iterations:     7.0000                                      
-----------------------------------------------------------------
               Coef.   Std.Err.     z      P>|z|   [0.025  0.975]
-----------------------------------------------------------------
last year      1.4803    0.0543   27.2503  0.0000  1.3738  1.5868
never          2.9410    0.0238  123.5514  0.0000  2.8944  2.9877
sometime       2.1395    0.0599   35.6884  0.0000  2.0220  2.2570

